### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
- [Model Architecture](#1)
- [Dataset](#2)
- [Training pipeline](#3)

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
# generating a random seed
torch.manual_seed(21)
torch.cuda.manual_seed(21)

# Defining Hyperparamethers
cuda = True
batch_size = 64


# importing the MNIST dataset
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:02<00:00, 3319731.62it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28862447.90it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3534298.93it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)

        # Fully Connected Layers
        self.fc1 = nn.Linear(64*7*7, 512) 
        self.bn5 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 1024)
        self.bn6 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, 10)

        # Dropout Layers
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = x.view(x.size(0), -1) # Flatten the tensor
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout2(x)

        x = F.relu(self.bn6(self.fc2(x)))
        x = self.dropout2(x)

        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [ ]:
model = Net()
# move the model to the GPU
if cuda: model.cuda()

In [5]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, momentum=args.momentum)

# train the network
def train():
    
    # log the training information so we can later visualise it using tensorboard
    tb = SummaryWriter('./log_pytorch')

    # put the model into training mode
    model.train()
    # start the training, which will run for several epochs (https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/)
    for epoch in range(1, args.epochs + 1):
        # keep track of the number of correctly classified samples during each epoch
        correct = 0
        # create our function to train the model
        # divide the dataset into batches (https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/)
        for batch_idx, (data, target) in enumerate(train_loader):
            # move the data to the GPU
            if args.cuda: data, target = data.cuda(), target.cuda()
            # convert the data to PyTorch variables (Tensor)
            #data, target = Variable(data), Variable(target)
            # initialise the gradients to zero
            optimizer.zero_grad()
            # feed the input data into the CNN model and get the output
            output = model(data)
            # calculate the error by comparing the result that should be obtained with the output produced by our model
            # here we use the negative log likelihood loss (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html)
            loss = F.nll_loss(output, target)
            # once we have measure the error, we apply back-propagation to calculate the gradients needed to update the model parameters
            # https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html
            loss.backward()
            # update the model parameters (weights) using SGD
            optimizer.step()
            # the predicted class is the one with the maximum probability in the posterior probability vector (output of softmax)
            pred = output.data.max(1)[1]
            # count the number of times the prediction made by our model is correct (i.e., equal to the target)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum() # M
    
            # print the current loss
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item())) 
                    
        # add train loss to tensorboard
        tb.add_scalar("epoch loss", loss.item(), epoch)
        # add accuracy to tensorboard
        tb.add_scalar("epoch accuracy", 100. * correct / len(train_loader.dataset), epoch)
        # add weight histogram to tensorboard
        for name, weight in model.named_parameters():
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f'{name}.grad',weight.grad, epoch)
            
    # save the model to a .pth file
    print('Saving CNN to %s' % args.model_path)
    torch.save(model.state_dict(), args.model_path)
    # add graph to tensorboard
    tb.add_graph(model, (data,))
        

# evaluate the model performance (accuracy) on the test data
def test():
    
    # load the model we trained and saved in args.model_path
    model.load_state_dict(torch.load(args.model_path))
    
    # put model into test mode
    model.eval()
    # initialise the variables used to track the loss and the correct predictions made by the model
    test_loss = 0
    correct = 0
    # fetch the test data
    for data, target in test_loader:
        if args.cuda: data, target = data.cuda(), target.cuda()
        # no need to compute gradients now, we're only "using" the CNN, we're not training it
        with torch.no_grad():
            # convert the data to PyTorch variables (Tensor)
            # data, target = Variable(data, volatile=True), Variable(target)
            # feed the input data into the CNN model and get the output
            output = model(data)
            # calculate the loss for this batch of data points and add it to the total
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # the predicted class is the one with the maximum probability in the posterior probability vector (output of softmax)
            pred = output.data.max(1)[1]
            # count the number of times the prediction made by our model is correct (i.e., equal to the target)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum() # M

    # calculate the average loss
    test_loss /= len(test_loader.dataset)
    # print the average loss and the percentage of correct classifications made by our model (i.e., accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100*correct / len(test_loader.dataset)))

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                 

In [ ]:
!python mnist_cnn_keras.py --mode 'test' --model_path 'models/mnist_cnn.keras'

In [ ]:
!python mni

In [7]:
### <font color='289C4E'>Table of contents<font><a class='anchor' id='top'></a>
- [Section 1](#1)
- [Section 2](#2)
- [Section 3](#3)

SyntaxError: invalid syntax (976892717.py, line 1)